In [1]:
#-*- coding:utf-8 -*-
import os
import os.path as osp
import time
from collections import defaultdict
from typing import DefaultDict, Dict, Set, TextIO, Tuple, Union, List
import numpy as np
import sys
from abc import *
import torch
 4r4rr bd
np.set_printoptions(threshold=sys.maxsize)

ModuleNotFoundError: No module named 'config'

In [66]:
###################################################
#   Abstract Graph class                          #
#                                                 #
###################################################
class DiGraph(metaclass=ABCMeta):
    
    @abstractmethod
    def __str__(self):
        pass
    
    @abstractmethod
    def nodes(self) -> Set[int]:
        pass

    @abstractmethod
    def edges(self) -> List[Tuple[int, int]]:
        pass
    
    @abstractmethod
    def add_vertex(self, v: int) -> None:
        pass
    
    @abstractmethod
    def remove_vertex(self, v: int) -> None:
        pass

    @abstractmethod
    def add_edge(self, src: int, dst: int) -> None:
        pass

    @abstractmethod
    def remove_edge(self, src: int, dst: int) -> None:
        pass

    @abstractmethod
    def out_neighbor(self, v: int):
        pass

    @abstractmethod
    def out_degree(self, v: int) -> int:
        pass

    @abstractmethod
    def has_edge(self, src: int, dst: int) -> bool:
        pass


# **Homework 1-1**

TODO : Implement following methods for graph network                                 
- edges()               : return list of edges [e1, e2, ...]               
- add_vertex(v)         : add vertex v 
- remove_vertex(v)      : remove vertex v 
- add_edge(src, dst)    : add edge from one vertex(src) to another(dst)             
- remove_edge(src, dst) : remove edge from src to dst                               
- out_neighbor(v)       : get neighbors of vertex(v)                                
- out_degree(v)         : get the number of edges from vertex v                     
- has_edge(src, dst)    : return True if the edge (src, dst) is in the graph       

In [67]:
###################################################
#   Dictionary Array Graph                        #
#    - 정점       : self._nodes                   #             
#    - 나가는간선 : self._out_neighbor            #
#                                                 #
###################################################
class Graph_dict(DiGraph):

    def __init__(self) -> None:
        self._nodes = set()
        self._out_neighbor = defaultdict(set)

    def __str__(self):
        result = ""
        for v in self._nodes:
            neighbors = self._out_neighbor[v]
            if not neighbors:
                result = result + str(v) + " : empty\n"
            else : result = result + str(v) + " : " +  str(neighbors) + "\n"
        return result
    
    def nodes(self) -> Set[int]:
        return self._nodes

    def edges(self) -> List[Tuple[int, int]]:
        edges = []
        for v1 in self._nodes:
            neighbors = self._out_neighbor[v1]
            for v2 in neighbors:
                edges.append((v1, v2))
        return edges
        


    ################# Write your answer here #####################
    def add_vertex(self, v: int) -> None:
        self._nodes.add(v)
        return

    ################# Write your answer here #####################
    def remove_vertex(self, v: int) -> None:
        self._nodes.discard(v)
        self._out_neighbor.pop(v, None)
        for node in self._nodes :
            self._out_neighbor[node].discard(v)

    def add_edge(self, src: int, dst: int) -> None:
        self.add_vertex(src)
        self.add_vertex(dst)
        self._out_neighbor[src].add(dst)
        return

    def remove_edge(self, src: int, dst: int) -> None:
        neighbor = self.out_neighbor(src)
        if dst in neighbor:
            self._out_neighbor[src].remove(dst)
        return 

    ################# Write your answer here #####################
    def out_neighbor(self, v: int) -> Set[int]:
        neighbor = set()
        if v in self._nodes :
            neighbor = self._out_neighbor[v]
        return neighbor
    
    ################# Write your answer here #####################
    def out_degree(self, v: int) -> int:
        degree = 0
        if v in self._nodes :
            degree = len(self.out_neighbor(v))
        return degree
    
    def has_edge(self, src: int, dst: int) -> bool:
        return dst in self._out_neighbor[src]
    
    ################# Write your answer here #####################
    # HW 1-2
    def subgraph(self, vertices: List[int]):
        res=Graph_dict()
        for v in vertices:
            res.add_vertex(v)
        for v in vertices:
            for v1 in vertices:
                if v==v2:
                    continue
                if self.has_edge(v,v1):
                    res.add_edge(v,v1)
        return res 


In [68]:
###################################################
#   Numpy Array Graph                             #
#    - 정점     : self._nodes                     #
#    - 간선     : self._data                      #
#    - max_size : self._max_size                  #
#                                                 #
###################################################
class Graph_numpy_array(DiGraph):
    def __init__(self, size: int) -> None:
        self._data = np.zeros((size, size))
        self._max_size = size
        self._nodes = set()

    def __str__(self):
        return str(self._data)
    
    def nodes(self) -> Set[int]:
        return self._nodes

    def edges(self) -> List[Tuple[int, int]]:
        edge_list = []
        for i in range(self._max_size):
            for j in range(self._max_size):
                if self._data[i][j] == 1:
                    edge_list.append((i, j))
        return edge_list

    def add_vertex(self, v: int) -> None:
        if v >= 0 and v < self._max_size:
            self._nodes.add(v)
        else:
            print("Error : vertex number is in the out of range")
    
    def remove_vertex(self, v: int) -> None:
        if v >= 0 and v < self._max_size:
            
            self._nodes.remove(v)
            self._data[v, :] = 0
            self._data[:, v] = 0
        else :
            print("Error : vertex number is in the out of range")

    ################# Write your answer here #####################
    def add_edge(self, src: int, dst: int) -> None:
        if src >= 0 and src < self._max_size and dst >= 0 and dst < self._max_size :
            self._nodes.add(src)
            self._nodes.add(dst)
            self._data[src][dst]=1
           
        return

    ################# Write your answer here #####################
    def remove_edge(self, src: int, dst: int) -> None:
        if src >= 0 and src < self._max_size and dst >= 0 and dst < self._max_size :
            self._data[src][dst]=0
        
        return


    def out_neighbor(self, v: int) -> Set[int]:
        return {x for x in range(self._max_size) if self._data[v][x] == 1}

    def out_degree(self, v: int) -> int:
        return len(self.out_neighbor(v))
    
    ################# Write your answer here #####################
    def has_edge(self, src: int, dst: int) -> bool:
        if src >= 0 and src < self._max_size and dst >= 0 and dst < self._max_size :
            if self._data[src][dst]==1:
                return True
            else : 
                return False
        else:
            return False

    def subgraph(self, vertices: List[int]):
        subgraph = Graph_numpy_array(self._max_size) 
        for v in vertices:
            subgraph.add_vertex(v)
        for v1 in vertices:
            for v2 in vertices:
                if v1 == v2:
                    continue
                elif self.has_edge(v1, v2):
                    subgraph.add_edge(v1, v2)
        return subgraph



In [69]:
from queue import PriorityQueue
from typing import DefaultDict
from collections import defaultdict
import math

# Breath First Search algorithm
def shortest_path_from(G: DiGraph, src: int) -> defaultdict(int):
    queue = []
    visited = {v: False for v in G.nodes()}
    distance = {v: float('inf') for v in G.nodes()}
    visited[src] = True
    distance[src] = 0
    queue.append(src)
    
    while (len(queue) != 0):
        v = queue.pop(0)
        neighbors = G.out_neighbor(v)
        for u in G.out_neighbor(v):
            if visited[u] is False:
                visited[u] = True
                distance[u] = distance[v] + 1
                queue.append(u)
    return distance

# **Test script for homework 1-1**

You must pass all of following assertions

In [70]:
graph_dict = Graph_dict()
graph_numpy_array = Graph_numpy_array(10)

# Test 1 : vertex addition
graph_dict.add_vertex(1)
graph_dict.add_vertex(3)
graph_numpy_array.add_vertex(1)
graph_numpy_array.add_vertex(3)
assert graph_dict.nodes() == {1, 3}
assert graph_numpy_array.nodes() == {1, 3}

# Test 2 : edge addition & has_edge
graph_dict.add_edge(1, 3)
graph_dict.add_edge(1, 2)
graph_dict.add_edge(2, 3)
graph_numpy_array.add_edge(1, 3)
graph_numpy_array.add_edge(1, 2)
graph_numpy_array.add_edge(2, 3)
node = graph_dict.nodes()

assert graph_dict.edges() == [(1, 2), (1, 3), (2, 3)]
assert graph_numpy_array.edges() == [(1, 2), (1, 3), (2, 3)]
assert graph_dict.has_edge(1, 2) == True
assert graph_numpy_array.has_edge(1, 2) == True
assert graph_dict.has_edge(3, 1) == False
assert graph_numpy_array.has_edge(3, 1) == False

# Test 3 : duplicated edges
graph_dict.add_edge(1, 3)
graph_numpy_array.add_edge(1, 3)

assert graph_dict.edges() == [(1, 2), (1, 3), (2, 3)]
assert graph_numpy_array.edges() == [(1, 2), (1, 3), (2, 3)]

graph_dict.add_edge(5, 9)
graph_numpy_array.add_edge(5, 9)
assert graph_dict.edges() == [(1, 2), (1, 3), (2, 3), (5, 9)]
assert graph_numpy_array.edges() == [(1, 2), (1, 3), (2, 3), (5, 9)]

# Test 4 : remove vertex
graph_dict.remove_vertex(2)
graph_numpy_array.remove_vertex(2)

assert graph_dict.nodes() == {1, 3, 5, 9}
assert graph_numpy_array.nodes() == {1, 3, 5, 9}
assert graph_dict.edges() == [(1, 3), (5, 9)]
assert graph_numpy_array.edges() == [(1, 3), (5, 9)]

# Test 5 : remove edge
graph_dict.add_edge(2, 3)
graph_dict.remove_edge(2, 3)
graph_numpy_array.add_edge(2, 3)
graph_numpy_array.remove_edge(2, 3)

assert graph_dict.has_edge(2, 3) == False
assert graph_numpy_array.has_edge(2, 3) == False

# Test 6 : neighborhood
graph_dict.add_edge(1, 4)
graph_dict.add_edge(1, 2)
graph_numpy_array.add_edge(1, 4)
graph_numpy_array.add_edge(1, 2)

assert graph_dict.out_neighbor(1) == {2, 3, 4}
assert graph_numpy_array.out_neighbor(1) == {2, 3, 4}

assert graph_dict.out_degree(1) == 3
assert graph_dict.out_degree(1) == 3

# Test 7 : shortest path
import sys
sys.path.append('..')
print(os.getcwd())
path='../data/hw/hw1/random.txt'
data = osp.abspath(osp.join(os.getcwd(), 'drive/MyDrive/data/hw/hw1/random.txt'))
graph_dict = Graph_dict()
graph_numpy_array = Graph_numpy_array(100)
f = open(path)
for line in f:
    print(line)
    v1, v2 = map(int, line.split())
    graph_dict.add_edge(v1, v2)
    graph_numpy_array.add_edge(v1, v2)

dist_dict = shortest_path_from(graph_dict, 1)
dist_numpy = shortest_path_from(graph_numpy_array, 1)

for key in dist_dict.keys():
	assert dist_dict[key] == dist_numpy[key]

assert dist_dict[1] == 0
assert dist_dict[17] == 2
assert dist_dict[16] == 4
assert dist_dict[30] == 3


c:\Users\songbae\BoostCamp\Day21
1 3

2 4

3 5

4 6

5 7

6 8

7 9

8 10

9 12

10 12

11 13

12 14

13 15

14 16

15 17

16 18

17 19

18 20

19 21

20 22

21 23

22 24

23 25

24 26

25 27

26 28

27 29

28 30

29 1

30 2

1 10

5 17

10 20

3 19

5 14

15 27

4 8

7 25

1 15

19 30

22 28

11 21

2 25

7 22

16 21

18 29

11 26

4 13

6 24

9 13

24 17




# **Homework 1-2**
                                                                    
TODO : Implement following methods for pagerank algorithm                           
1) Graph_dict   
First, you implement subgraph methods in the graph_dict class 

- subgraph(vertices) : get the induced subgraph containing the nodes and edges

2) pagerank(graph, damping_factor, maxiters, tol)   

In [76]:
# Compute the distance between two dictionaries based on L1 norm
import pprint
def l1_distance(x: DefaultDict[int, float], y: DefaultDict[int, float]) -> float:
    err: float = 0.0
    for k in x.keys():
        err += abs(x[k] - y[k])
    return err

################################################################################
# Run the pagerank algorithm iteratively using the graph_dict                  #
#  parameters                                                                  #
#    - graph : Directed graph (graph_dict object)                              #
#    - damping_factor : Damping factor                                         #
#    - maxiters : The maximum number of iterations                             #
#    - tol : Tolerance threshold to check the convergence                      #
################################################################################
def pagerank(
    graph: Graph_dict,
    damping_factor: float,
    maxiters: int,
    tol: float,
) -> Dict[int, float]:
    vec: DefaultDict[int, float] = defaultdict(float)  # Pagerank vector
    r_new:DefaultDict[int,float]=defaultdict(float)

    ############### TODO: Implement the pagerank algorithm #####################    
    r_old:DefaultDict[int,float]=defaultdict(float)
    for v in graph.nodes():
      r_old[v]=1/len(graph.nodes())
    
    for itr in range(maxiters):
        S=0
        for j in graph.nodes():
            r_new[j]=0
            for i in graph.nodes():
                if graph.has_edge(i,j) and graph.out_degree(i)!=0:
                    r_new[j]+=damping_factor*r_old[i]/graph.out_degree(i)
            S+=r_new[j]
        
        for j in graph.nodes():
            r_new[j]+=(1-S)/len(graph.nodes())

        #### Check the convergence ###

        # Stop the iteration if L1norm[PR(t) - PR(t-1)] < tol
        delta: float = 0.0
        delta = l1_distance(r_new, r_old)

        print(f"[Iter {itr}]\tDelta = {delta}")

        if delta < tol:
            break
        for v in graph.nodes():
            r_old[v]=r_new[v]


    ########################### Implementation end #############################

    return dict(r_new)



You can test your pagerank algorithm using following code. (same as lab3)

In [72]:
G = Graph_dict()
import pprint
path='../data/hw/hw1/'
path_data='../data/others/'
path_v2n = osp.abspath(osp.join(path_data, 'vertex2name.txt'))
path_edges = osp.abspath(osp.join(path_data, 'edges.txt'))

# keyword : deep_learning.txt (딥러닝), lee.txt (이순신), bong.txt(봉준호)
path_keyword = osp.abspath(osp.join(path, 'deep_learning.txt'))

f = open(path_edges)
for line in f:
    v1, v2 = map(int, line.split())
    G.add_edge(v1, v2)                                                 
                                 
n2v = {}
v2n = {}
f = open(path_v2n, encoding='UTF8')
for line in f:
    v, n = line.split()
    v = int(v)
    n = n.rstrip()
    n2v[n] = v
    v2n[v] = n

node_key = []
f = open(path_keyword)
for line in f:
    v = line.rstrip()
    v = int(v)
    node_key.append(v)

H = G.subgraph(node_key)

print("###### PageRank Algorithm ######")
pr = pagerank(H, damping_factor = 0.9 , maxiters = 100, tol = 1e-06)
res = [key for (key, value) in sorted(pr.items(), key=lambda x:x[1], reverse=True)]
res_value = [value for (key, value) in sorted(pr.items(), key=lambda x:x[1], reverse=True)]
for item in res[:10]:
    pprint.pprint(v2n[item])


''' 
Expected Output (solution) 

###### PageRank Algorithm ######
[Iter 0]	Delta = 0.2556213017751478
[Iter 1]	Delta = 0.03539371870732816
[Iter 2]	Delta = 0.004900668744091563
[Iter 3]	Delta = 0.0006785541337973044
[Iter 4]	Delta = 9.395364929493111e-05
[Iter 5]	Delta = 1.3008966825434065e-05
[Iter 6]	Delta = 1.8012415605173082e-06
[Iter 7]	Delta = 2.494026775656799e-07
딥러닝
PyTorch
OpenCV
라온피플
이스트소프트
인공지능인문학
자동긴급제동장치
케플러-90i
T2d
심심이

'''


###### PageRank Algorithm ######
[Iter 0]	Delta = 0.5964497041420118
[Iter 1]	Delta = 0.5964497041420118
[Iter 2]	Delta = 0.5964497041420118
[Iter 3]	Delta = 0.5964497041420118
[Iter 4]	Delta = 0.5964497041420118
[Iter 5]	Delta = 0.5964497041420118
[Iter 6]	Delta = 0.5964497041420118
[Iter 7]	Delta = 0.5964497041420118
[Iter 8]	Delta = 0.5964497041420118
[Iter 9]	Delta = 0.5964497041420118
[Iter 10]	Delta = 0.5964497041420118
[Iter 11]	Delta = 0.5964497041420118
[Iter 12]	Delta = 0.5964497041420118
[Iter 13]	Delta = 0.5964497041420118
[Iter 14]	Delta = 0.5964497041420118
[Iter 15]	Delta = 0.5964497041420118
[Iter 16]	Delta = 0.5964497041420118
[Iter 17]	Delta = 0.5964497041420118
[Iter 18]	Delta = 0.5964497041420118
[Iter 19]	Delta = 0.5964497041420118
[Iter 20]	Delta = 0.5964497041420118
[Iter 21]	Delta = 0.5964497041420118
[Iter 22]	Delta = 0.5964497041420118
[Iter 23]	Delta = 0.5964497041420118
[Iter 24]	Delta = 0.5964497041420118
[Iter 25]	Delta = 0.5964497041420118
[Iter 26]	Delta

' \nExpected Output (solution) \n\n###### PageRank Algorithm ######\n[Iter 0]\tDelta = 0.2556213017751478\n[Iter 1]\tDelta = 0.03539371870732816\n[Iter 2]\tDelta = 0.004900668744091563\n[Iter 3]\tDelta = 0.0006785541337973044\n[Iter 4]\tDelta = 9.395364929493111e-05\n[Iter 5]\tDelta = 1.3008966825434065e-05\n[Iter 6]\tDelta = 1.8012415605173082e-06\n[Iter 7]\tDelta = 2.494026775656799e-07\n딥러닝\nPyTorch\nOpenCV\n라온피플\n이스트소프트\n인공지능인문학\n자동긴급제동장치\n케플러-90i\nT2d\n심심이\n\n'

# **Test script for homework 1-2**

In [77]:
# Test 1 : subgraph
import networkx as nx

G = Graph_dict()
G2 = Graph_numpy_array(100)
for i in range(0, 10):
    G.add_edge(i, i+1)
    G2.add_edge(i, i+1)

H = G.subgraph([1, 2, 3, 4, 5])
H2 = G2.subgraph([1, 2, 3, 4, 5])
print(H.edges())
assert H.edges() == [(1, 2), (2, 3), (3, 4), (4, 5)]
assert H.edges() == H2.edges()

H = G.subgraph([1, 3, 5])
H2 = G2.subgraph([1, 3, 5])
assert H.edges() == []
assert H.edges() == H2.edges()

H = G.subgraph(G.nodes())
H2 = G2.subgraph(G.nodes())
assert H.edges() == [(0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 8), (8, 9), (9, 10)]
assert H.edges() == H2.edges()
assert H.edges() == G.edges()

# Test 2 : PageRank
G = Graph_dict()
G_nx = nx.DiGraph()
data = osp.abspath(osp.join(path, 'ego-facebook.txt'))

f = open(data)
for line in f:
    v1, v2 = map(int, line.split())
    G.add_edge(v1, v2)
    G_nx.add_edge(v1, v2)


pr = nx.pagerank(G_nx, alpha = 0.9)
nx_res = [key for (key, value) in sorted(pr.items(), key=lambda x:x[1], reverse=True)]


pr = pagerank(G, damping_factor = 0.9, maxiters = 100, tol = 1e-06)
our_res = [key for (key, value) in sorted(pr.items(), key=lambda x:x[1], reverse=True)]

assert nx_res[:10] == our_res[:10]


'''
Expected output

[Iter 0]	Delta = 0.698921186626955
[Iter 1]	Delta = 0.3389690628171277
[Iter 2]	Delta = 0.17288562623997814
[Iter 3]	Delta = 0.09077429613251603
[Iter 4]	Delta = 0.04646162762095549
[Iter 5]	Delta = 0.022900195846917264
[Iter 6]	Delta = 0.011861585170685194
[Iter 7]	Delta = 0.006383508296346828
[Iter 8]	Delta = 0.0032048867134547676
[Iter 9]	Delta = 0.0015812728529011953
[Iter 10]	Delta = 0.000850118589198698
[Iter 11]	Delta = 0.0004365949887159213
[Iter 12]	Delta = 0.00021339393793977216
[Iter 13]	Delta = 0.0001085382627142086
[Iter 14]	Delta = 5.853815016438714e-05
[Iter 15]	Delta = 2.9449618139342482e-05
[Iter 16]	Delta = 1.4482341812984927e-05
[Iter 17]	Delta = 7.728298696144863e-06
[Iter 18]	Delta = 3.9759243921904535e-06
[Iter 19]	Delta = 1.9449675092040128e-06
[Iter 20]	Delta = 9.89687162248964e-07

'''

[(1, 2), (2, 3), (3, 4), (4, 5)]
[Iter 0]	Delta = 0.6989211866269575
[Iter 1]	Delta = 0.3389690628171291
[Iter 2]	Delta = 0.17288562623997883
[Iter 3]	Delta = 0.09077429613251968
[Iter 4]	Delta = 0.04646162762095538
[Iter 5]	Delta = 0.022900195846917916
[Iter 6]	Delta = 0.01186158517068688
[Iter 7]	Delta = 0.00638350829634752
[Iter 8]	Delta = 0.003204886713450502
[Iter 9]	Delta = 0.0015812728528990796
[Iter 10]	Delta = 0.0008501185891999062
[Iter 11]	Delta = 0.00043659498870927775


KeyboardInterrupt: 